In [1]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [1]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v5\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [4]:
print(len(val_uids))

2715


In [3]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'torch' is not defined

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v5/0707a_preprocess.yaml -m ./config/Ego4D_final_v5/0707a_model.yaml -t ./config/Ego4D_final_v5/0707a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [8]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v5/0707a_preprocess.yaml', model_config='./config/Ego4D_final_v5/0707a_model.yaml', train_config='./config/Ego4D_final_v5/0707a_train.yaml')
Prepare training ...


In [9]:
ckpt_path = r'./output/0707a/ckpt/Ego4D_final_v5/1200000.pth.tar'
ckpt = torch.load(ckpt_path)

In [10]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=True)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Using speaker embeddings.
True
2
=> Using VarianceAdaptorWithSpeaker.
Number of FastSpeech2 Parameters: 35165553


In [11]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [12]:
len(dataset)

2715

In [13]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [14]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [15]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0707a/result/Ego4D_final_v5\\plots'

In [16]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [17]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/2715 [00:00<?, ?it/s]

# MCD

In [4]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [19]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v5\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [20]:
len(val_uids)

2715

In [21]:
recon_folder = r'.\output\0707a\result\Ego4D_final_v5\wav\reconstructed'
syn_folder = r'.\output\0707a\result\Ego4D_final_v5\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v5\val\Ego4D_final_v5'

In [22]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [23]:
%%time
uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_recon.append(result)

  0%|          | 0/2715 [00:00<?, ?it/s]

CPU times: total: 10min 27s
Wall time: 1min 44s


In [24]:
%%time
uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_gt.append(result)

  0%|          | 0/2715 [00:00<?, ?it/s]

CPU times: total: 11min 19s
Wall time: 1min 50s


In [25]:
v5_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [26]:
v5_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0707a_1M2.csv")

In [27]:
print(f"MCDs_recon mean on 0707a_1M2: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0707a_1M2: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0707a_1M2: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0707a_1M2: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0707a_1M2: 15.63803768157959
MCDs_recon std on 0707a_1M2: 2.9097180366516113
MCDs_gt mean on 0707a_1M2: 16.158737182617188
MCDs_gt std on 0707a_1M2: 2.9617269039154053


In [28]:
v5_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
2370,0e4ae9e1-640b-45d1-90b2-17b4e941322d,7.243025,10.131522
107,a6d4dbc4-969d-4819-af85-f3b3cb56195e,7.783217,13.009296
444,af8de660-f916-4a2c-92ac-4e767436ed2c,8.237475,8.663778
373,1b66fe95-6176-4ee1-98ba-984c9bfe1f5f,8.497393,8.395654
1749,2667f8d5-30e0-44da-80ce-27df506a8a82,8.592734,10.184369


In [29]:
v5_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
1625,c2b61a6a-1e3f-4ab7-a207-c8e748b4d330,30.934711,26.058529
68,898b49a9-5b65-49f8-bbca-c0f784f80b98,28.250639,15.672288
2632,dc0bb287-5310-44a9-b491-b195d98fd53e,27.944120,20.941483
1866,3ac7d90c-3ef3-4dbe-b818-dd2583a18070,27.534554,26.702335
555,ec1ee52c-587c-4f8d-8760-52e6ba6019d3,26.466930,25.490271


# inference on train

In [5]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v5/0707a_preprocess.yaml -m ./config/Ego4D_final_v5/0707a_model.yaml -t ./config/Ego4D_final_v5/0707a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [8]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v5/0707a_preprocess.yaml', model_config='./config/Ego4D_final_v5/0707a_model.yaml', train_config='./config/Ego4D_final_v5/0707a_train.yaml')
Prepare training ...


In [9]:
ckpt_path = r'./output/0707a/ckpt/Ego4D_final_v5/1200000.pth.tar'
ckpt = torch.load(ckpt_path)

In [10]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=True)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Using speaker embeddings.
True
2
=> Using VarianceAdaptorWithSpeaker.
Number of FastSpeech2 Parameters: 35165553


In [11]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "train.txt", 'train', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [12]:
len(dataset)

28010

In [13]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [14]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [15]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'train', 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'train', 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'train', 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'train', 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'train', 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [16]:
# for batchs in tqdm(loader):
#     for targets in batchs:
#         targets = to_device(targets, device)
#         with torch.no_grad():
#             predictions = model(*(targets[2:]))
#         basenames = targets[0]
#         for i in range(len(predictions[0])):
#             basename = basenames[i]
#             src_len = predictions[8][i].item()
#             mel_len = predictions[9][i].item()
#             mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
#             mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

#             torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
#             torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
#             duration = predictions[5][i, :src_len].detach().cpu().numpy()
#             if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
#                 pitch = predictions[2][i, :src_len].detach().cpu().numpy()
#                 pitch = expand(pitch, duration)
#             else:
#                 pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
#             if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
#                 energy = predictions[3][i, :src_len].detach().cpu().numpy()
#                 energy = expand(energy, duration)
#             else:
#                 energy = predictions[3][i, :mel_len].detach().cpu().numpy()

#             with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
#                                    "stats.json")) as f:
#                 stats = json.load(f)
#                 stats = stats["pitch"] + stats["energy"][:2]
                                       
#             fig = plot_mel(
#                 [
#                     (mel_prediction.cpu().numpy(), pitch, energy),
#                     (mel_target.cpu().numpy(), pitch, energy),
#                 ],
#                 stats,
#                 ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
#             )
#             ### TODO: change to svg
#             plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
#             plt.close()

#         # from .model import vocoder_infer

#         mel_predictions = predictions[1].transpose(1, 2)
#         mel_targets = targets[6].transpose(1, 2)
        
#         lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
#         wav_predictions = vocoder_infer(
#             mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
#         )
#         wav_targets = vocoder_infer(
#         mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
#     )
    
#         sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
#         for wav, basename in zip(wav_predictions, basenames):
#             wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
#         for wav, basename in zip(wav_targets, basenames):
#             wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
#     #     break
#     # break

In [17]:
for idx, batchs in tqdm(enumerate(loader)):
    if idx < 27760:
        continue
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

0it [00:00, ?it/s]

In [19]:
print(11111)

11111


# MCD on train

In [20]:
split_txt_train_path = r'.\preprocessed_data\Ego4D_final_v5\train.txt'
train_uids = []
with open(split_txt_train_path) as file:
    for line in file:
        # print(line.split('|')[0])
        train_uids.append(line.split('|')[0])

In [21]:
len(train_uids)

28010

In [22]:
recon_folder = r'.\output\0707a\result\Ego4D_final_v5\train\wav\reconstructed'
syn_folder = r'.\output\0707a\result\Ego4D_final_v5\train\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v5\train\Ego4D_final_v5'

In [23]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [24]:
%%time
uids = []
MCDs_recon = []
for uid in tqdm(train_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_recon.append(result)

  0%|          | 0/28010 [00:00<?, ?it/s]

CPU times: total: 2h 9min 59s
Wall time: 21min 34s


In [25]:
%%time
uids = []
MCDs_gt = []
for uid in tqdm(train_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_gt.append(result)

  0%|          | 0/28010 [00:00<?, ?it/s]

CPU times: total: 2h 16min 37s
Wall time: 23min 35s


In [26]:
v5_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [27]:
v5_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0707a_1M2_train.csv")

In [28]:
print(f"MCDs_recon mean on 0707a_1M2_train: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0707a_1M2_train: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0707a_1M2_train: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0707a_1M2_train: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0707a_1M2_train: 9.539996147155762
MCDs_recon std on 0707a_1M2_train: 1.7698731422424316
MCDs_gt mean on 0707a_1M2_train: 11.003610610961914
MCDs_gt std on 0707a_1M2_train: 2.176541566848755


In [29]:
v5_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
17652,36942da8-0899-4eb9-97a0-00832dc652cc,3.679867,13.465104
13537,6e3331e3-01dd-4b4e-9718-6877ca793ca3,3.785106,16.777239
9348,009e6bb2-3d56-4644-8c4a-09503518a22c,3.970977,11.511535
123,78bed0db-eaa5-4bdf-b583-e3306e7d8f61,3.979556,9.241287
24610,3041a0da-8b32-43a8-b8fa-4c2674df353a,4.017649,12.608796


In [30]:
v5_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
1841,55b72a13-9e0a-4e0e-9289-fd14ab24e627,21.148561,20.978930
1318,797cc95a-9062-4bc6-b28d-d3100a6ad817,20.886789,17.400734
1875,b69e0da9-dd12-4394-8727-c75a3c15d9fb,20.741346,20.559359
1425,a40fce22-6afc-42b6-9101-306fd14ca3d6,19.676283,21.359287
1488,5248ebe3-4905-4799-82b2-410205c8509a,19.167067,19.473600
